In [ ]:
HTML(read(open("style.css"), String))

# Quiescence Search

Quiescence Search is a search algorithm designed to avoid the horizon effect in the game of chess. It is called when the search function (e.g. Minimax, alpha-beta-pruning,...) have reached their maximum depth. Quiescence Search will then further search through the game tree until it hits a quiet position. Then and only then it will use the heuristic function to estimate the position. It is important to not stop searching when hitting the end of the depth as this last move can conclude into very good moves for the opposing side. As an example the last move searched by the algorithm is a queen capturing a pawn. The evaluation function therefore will return a score of +100 centipawns as it just took this pawn. Notice that the search algorithm will not see that this pawn was protected resulting in the queen being captured in the move after. This issue is called the **horizon effect**.

Quiescence Search will therefore keep calculate all tactical moves and stop when it hits a quiet position. As only the tactical moves will be evaluated, this evaluation will not be taking as much performance as a regular search. Additionally Pruning will result in about 50-90% of paths pruned. (https://www.chessprogramming.org/Quiescence_Search)

There are many different approaches on implementing Quiescence Search. The definition of a quiet position also varies. Therefore we define a quiet position as follows:

A position is quiet iff: `there are no captures (en passant included)` ∨ `there are no available checks` ∨ `there are no promotions` 

In [ ]:
using Pkg
# Pkg.add("Chess")
using Chess
using NBInclude
using BenchmarkTools

[Chess Programming Wiki "Quiescence Search"](https://www.chessprogramming.org/Quiescence_Search)

In [ ]:
@nbinclude("Memoization.ipynb")

## Auxiliary functions

### Function: hasCaptureMoves
This function takes in a chess game state and returns a boolean indicating whether there are any capture moves available for the opponent.

Arguments:
1. `State::Board` A chess board in the current state.

Returns:
1. `true` if there are any capture moves available for the opponent, `false` otherwise.

In [ ]:
function hasCaptureMoves(State::Board)
    allPiecesSq = pieces(State, coloropp(sidetomove(State)))
    return any(isattacked(State, square, coloropp(sidetomove(State))) for square in allPiecesSq)
end

### Function: isTacticalMove
The function `isTacticalMove` checks whether the move done on a state is a tactical move. 

A move is a tactical move if is one of the following is true:
1. The move is capturing a piece
1. The move is promoting a pawn
1. The move is an en passant capture
1. The move checks the opposing king

Arguments:

1. `State::Board` The current board
1. `move::Move`   The move on the `State` that should be checked for tactical moves

Returns:

1. `true` if the move is a tactical move, otherwise false

In [ ]:
function isTacticalMove(State::Board, move::Move)
    if pieceon(State, to(move)) != EMPTY || ispromotion(move) || epsquare(State) == to(move)
        return true
    end
    undoinfo = domove!(State, move)
    isNextMoveCheck = ischeck(State)
    undomove!(State, undoinfo)
    return isNextMoveCheck
end

In [ ]:
## function currently not used. TODO: Check whether this function is useful? bad performance
function isQuiet(State)
    return any(isTacticalMove(State, move) for move in moves(State))
end

## General Quiescence Search

### First version: quiescenceNoPrune

The function `quiescenceNoPrune` uses the quiescence algorithm to determine the best score. It only stops searching if the position is quiet and only searches through positions which have tactical moves. This function does not contain any pruning attempts and therefore has a very long runtime.

Arguments:

1. `State::Board` The current board
1. `score::Int64` The score of the current board
1. `hash::UInt64` The hash of the current board

Returns:

1. `bestScore::Int64` The best score that can be achieved for the current State if both sides play optimally.

In [ ]:
## not used, because of bad performance
function quiesceNoPrune(State, score, hash)
    bestscore = score
    for move in moves(State)
        if isTacticalMove(State, move)
            nextScore, nextHash = updateBoardData(State, score, hash, move)
            # nextBoard = domove(State, move)
            undoinfo = domove!(State, move)
            # value = -evaluate(State, quiesce, nextScore, nextHash, depth-1, -beta, -alpha)
            value = quiesceNoPrune(State, nextScore, nextHash)
            undomove!(State, undoinfo)
            bestscore = sidetomove(State) == WHITE ? max(bestscore, value) : min(bestscore, value)
        end
    end
    
    return bestscore
end

The function `quiescence` uses the quiescence algorithm to determine the best score. It only stops searching if the position is quiet and only searches through positions which have tactical moves. This function is often implemented in literature but implements a version fitting the `Negamax` function. As this project uses `Alpha-Beta-Pruning` and therefore has a different evaluation method this function cannot be used.

In [ ]:
## NegaMax version of Quiescence: not used
function quiesce(State, score, hash, depth, alpha, beta)
    display(State)
    println("$(score), $(depth), $(alpha), $(beta)")
    value = score
    if value >= beta
        println("Pruned 1")
        return beta
    end
    if value > alpha
        alpha = value
    end
    for move in moves(State)
        if isTacticalMove(State, move)
            nextScore, nextHash = updateBoardData(State, score, hash, move)
            
            undoinfo = domove!(State, move)
            
            # value = -evaluate(State, quiesce, nextScore, nextHash, depth-1, -beta, -alpha)
            value = -quiesce(State, nextScore, nextHash, depth-1, -beta, -alpha)
            
            println(value)
            undomove!(State, undoinfo)
            
            if value >= beta
                return beta
            end
            if value > alpha
                alpha = value
            end
        end
    end
    return alpha
end

## Quiescence Min and Max

The function `quiescenceMax` uses the quiescence algorithm to determine the best score (meaning maximizing the score) for positions where white moves. It only stops searching if the position is quiet and only searches through positions which have tactical moves.

Arguments:

1. `State::Board` The current board
1. `score::Int64` The score of the current board
1. `hash::UInt64` The hash of the current board
1. `depth::Int64` The depth of the search (initially always set to 0 and decremented by each step)
1. `alpha::Int64` The lower boundary of the score for this search
1. `beta::Int64`  The upper boundary of the score for this search

Returns:

1. `score::Int64` The best score that can be achieved for the current State if both sides play optimally.

In [ ]:
function quiesceMax(aBoard::AdvBoard, depth::Int64, 
                    cache::Dict{UInt64, Tuple{String, Int64, Int64}}, alpha::Int64, beta::Int64)::Int64
    value::Int64 = aBoard.score
    value >= beta && return beta
    alpha = max(alpha, value)
    
    for move in moves(aBoard.state)
        isTacticalMove(aBoard.state, move) || continue
        undoinfo::Tuple{Int64, UInt64, UndoInfo} = domoveAdv!(aBoard, move)
        value = evaluate(aBoard, quiesceMin, depth-1, cache, alpha, beta)
        # value = quiesceMin(State, nextScore, nextHash, depth-1, alpha, beta)
        undomoveAdv!(aBoard, undoinfo)
        value >= beta && return beta
        alpha = max(alpha, value) 
    end
    return alpha
end

The function `quiescenceMin` uses the quiescence algorithm to determine the best score (meaning minimizing the score) for positions where black moves. It only stops searching if the position is quiet and only searches through positions which have tactical moves.

Arguments:

1. `State::Board` The current board
1. `score::Int64` The score of the current board
1. `hash::UInt64` The hash of the current board
1. `depth::Int64` The depth of the search (initially always set to 0 and decremented by each step)
1. `alpha::Int64` The lower boundary of the score for this search
1. `beta::Int64`  The upper boundary of the score for this search

Returns:

1. `score::Int64` The best score that can be achieved for the current State if both sides play optimally.

In [ ]:
function quiesceMin(aBoard::AdvBoard, depth::Int64, 
                    cache::Dict{UInt64, Tuple{String, Int64, Int64}}, alpha::Int64, beta::Int64)::Int64
    value = aBoard.score
    value <= alpha && return alpha
    beta = min(value, beta)
    
    for move in moves(aBoard.state)
        isTacticalMove(aBoard.state, move) || continue
        undoinfo::Tuple{Int64, UInt64, UndoInfo} = domoveAdv!(aBoard, move)
        value = evaluate(aBoard, quiesceMax, depth-1, cache, alpha, beta)
        # value = quiesceMax(State, nextScore, nextHash, depth-1, alpha, beta)
        undomoveAdv!(aBoard, undoinfo)
        value <= alpha && return alpha
        beta = min(value, beta)
    end
    return beta
end

## Quiescence Search with see() function

The function `quiesceSee` uses the `see()` function from `Chess.jl` to determine an estimate whether any pieces may be captured in the near future. As the see function returns a value that represents the difference in value in material point. Therefore this value is multiplied by 100 as this roughly delivers the centipawn value.

Arguments:

1. `State::Board` The current board
1. `score::Int64` The score of the current board
1. `alpha::Int64` The lower boundary of the score for this search
1. `beta::Int64`  The upper boundary of the score for this search

Returns:

1. `score::Int64` The best score that can be achieved for the current State if both sides play optimally.

In [ ]:
function quiesceSee(State::Board, score::Int64, alpha::Int64, beta::Int64)
    if sidetomove(State) == WHITE
        bestEstimate::Int64 = -200000
        for move in moves(State)
            bestEstimate = max(bestEstimate, see(State, move))
        end
    else 
        bestEstimate = 200000
        for move in moves(State)
            bestEstimate = min(bestEstimate, see(State, move))
        end
    end

    return score + bestEstimate * 100
end

## Strategic Quiescence Search

In [ ]:
function evalPlus(State)
    if isCheck(State)
        return Inf
    end
    if hasCaptureMoves(State) && ispromotion(State)
    end
end

In [ ]:
function strategicQS(State, score, hash, alpha, beta)
    value = score
    if value >= beta
        return beta
    end
    if value > alpha
        alpha = value
    end
    
    for move in moves(State)
        nextScore, nextHash = updateBoardData(State, score, hash, move)
        undoinfo = domove!(State, move)
        evalPlusValue = evalPlus(State)
        if evalPlusValue > alpha
            actVal = -strategicQS(State, nextScore, nextHash, -beta, -alpha)
            if actVal > value
                value = actVal
                if value >= beta
                    return beta
                end
                if value > alpha
                    alpha = value
                end
            end
        elseif evalPlusValue > value
            value = evalPlusValue
        end
        undomove!(State, move)
    end
    return value
end